# Úkol č. 4 - regrese
**Deadline úkolu je uveden na [course pages](https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html).**

  * Cílem tohoto úkolu je vyzkoušet si řešit regresní problém na reálných datech.
  
> **Nejdůležitější na úkolu je to, abyste udělali vše procesně správně: korektní rozdělení datasetu, ladění hyperparametrů, vyhodnocení výsledků atp.**

## Dataset

  * Zdrojem dat je soubor `LifeExpectancyData.csv` na course pages (originál zde: https://www.kaggle.com/kumarajarshi/life-expectancy-who).
  * Popis datasetu najdete na uvedené stránce s originálem datasetu.
  * Cílová (vysvětlovaná) proměnná se jmenuje `Life expectancy `.
  

## Pokyny k vypracování
Body zadání, za jejichž (poctivé) vypracování získáte 12 bodů:

  1. Odeberte z dat body u kterých neznáte vysvětlovanou proměnnou.
  1. Rozdělte data na trénovací a testovací množinu.
  1. Proveďte základní průzkum dat. Na jeho základě adekvátně reagujte na problematické věci v datech (chybějící hodnoty, atd.).
  1. Aplikujte lineární a hřebenovou regresi a výsledky řádně vyhodnoťte:
    * K měření chyby použijte `mean_absolute_error`.
    * Experimentujte s tvorbou nových příznaků (na základě těch dostupných).
    * Experimentujte se standardizací/normalizací dat.
    * Vyberte si hyperparametry modelů k ladění a najděte jejich nejlepší hodnoty.
  1. Použijte i jiný model než jen lineární a hřebenovou regresi.


## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte tento Jupyter Notebook.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [135]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.impute import KNNImputer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

random_seed = 731

In [136]:
def clean(df):
    df.rename(columns={'Life expectancy ':'Life expectancy', 
    'Measles ' : 'Measles',
    ' BMI ' : 'BMI',
    'under-five deaths ' : 'under-five deaths',
    'Diphtheria ' : 'Diphtheria',
    ' HIV/AIDS' : 'HIV/AIDS',
    ' thinness  1-19 years' : 'thinness  1-19 years',
    ' thinness 5-9 years' : 'thinness 5-9 years'
    }, inplace=True)
    
    # remove rows which do not have explained variable
    df.drop(df[df['Life expectancy'].isna()].index, inplace=True)

    df['Status'] = df['Status'].apply(lambda x: 1 if x == 'Developed' else 0)
    return df

In [137]:
def rename(df):
    label_encoder = LabelEncoder()
    for col in df.select_dtypes('object').columns:
            df[col] = df[col].fillna('NaN')
            df[col] = label_encoder.fit_transform(df[col])
    return df

In [138]:
def scale(df):
        return pd.DataFrame(scaler.transform(df),indedf=df.indedf, columns=df.columns)

In [139]:
df = pd.read_csv('led.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'led.csv'

In [ ]:
display(df.describe())
display(df.isna().sum(axis=0))

In [ ]:
df = rename(df)
df = clean(df)
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['Life expectancy']), df['Life expectancy'], test_size=0.2, random_state=random_seed
)

In [ ]:
models = [LinearRegression, Ridge, KNeighborsRegressor, DecisionTreeRegressor]

In [ ]:
for classifier in models:
    model = classifier()
    scaler = StandardScaler()
    # fit and transform scaler and imputer on train data
    X = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index, columns=X_train.columns)
    # using KNNImputer for predicting missing values, should use hyperparameter tuning
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    X = pd.DataFrame(imputer.fit_transform(X),index=X.index, columns=X.columns)
    # fit model on train data
    model.fit(X, y_train)
    # transform test dataset with fitted scaler, imputer
    X = pd.DataFrame(scaler.transform(X_test),index=X_test.index, columns=X_test.columns)
    X = pd.DataFrame(imputer.transform(X),index=X.index, columns=X.columns)
    # predict
    pred = model.predict(X)
    error = np.sqrt(mean_absolute_error(y_test, pred))   
    print(classifier.__name__)
    print('Error: ', error)